In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os, time, pickle
from tqdm.notebook import tqdm

from tbpp_model import TBPP512, TBPP512_dense, TBPP512_dense_separable
from tbpp_utils import PriorUtil
from ssd_data import InputGenerator
from tbpp_training import TBPPFocalLoss

from utils.model import load_weights
from utils.training import MetricUtility

### Data

In [ ]:
from data_synthtext import GTUtility
with open('gt_util_synthtext_seglink.pkl', 'rb') as f:
    gt_util = pickle.load(f)

gt_util_train, gt_util_val = gt_util.split(0.9)

### Model

In [ ]:
# TextBoxes++
model = TBPP512(softmax=False)
weights_path = './models/ssd512_voc_weights_fixed.hdf5'
freeze = ['conv1_1', 'conv1_2',
          'conv2_1', 'conv2_2',
          'conv3_1', 'conv3_2', 'conv3_3',
          #'conv4_1', 'conv4_2', 'conv4_3',
          #'conv5_1', 'conv5_2', 'conv5_3',
         ]
batch_size = 24
experiment = 'tbpp512fl_synthtext'

In [ ]:
# TextBoxes++ with DSOD backbone
model = TBPP512_dense(softmax=False)
weights_path = None
freeze = []
batch_size = 6
experiment = 'dsodtbpp512fl_synthtext'

In [ ]:
# TextBoxes++ with dense blocks and separable convolution
model = TBPP512_dense_separable()
weights_path = None
freeze = []
batch_size = 8
experiment = 'dstbpp512fl_synthtext'

In [ ]:
prior_util = PriorUtil(model)

if weights_path is not None:
    load_weights(model, weights_path)

### Training

In [ ]:
epochs = 100
initial_epoch = 0

#optimizer = tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, decay=0, nesterov=True)
optimizer = tf.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0)

loss = TBPPFocalLoss(lambda_conf=10000.0, lambda_offsets=1.0)

#regularizer = None
regularizer = keras.regularizers.l2(5e-4) # None if disabled

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=False)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=False)


dataset_train, dataset_val = gen_train.get_dataset(), gen_val.get_dataset()
iterator_train, iterator_val = iter(dataset_train), iter(dataset_val)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment

if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

print(checkdir)

for l in model.layers:
    l.trainable = not l.name in freeze
    if regularizer and l.__class__.__name__.startswith('Conv'):
        model.add_loss(lambda l=l: regularizer(l.kernel))

metric_util = MetricUtility(loss.metric_names, logdir=checkdir)

@tf.function
def step(x, y_true, training=False):
    if training:
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            metric_values = loss.compute(y_true, y_pred)
            total_loss = metric_values['loss']
            if len(model.losses):
                total_loss += tf.add_n(model.losses)
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    else:
        y_pred = model(x, training=True)
        metric_values = loss.compute(y_true, y_pred)
    return metric_values

for k in tqdm(range(initial_epoch, epochs), 'total', leave=False):
    print('\nepoch %i/%i' % (k+1, epochs))
    metric_util.on_epoch_begin()

    for i in tqdm(range(gen_train.num_batches//4), 'training', leave=False):
        x, y_true = next(iterator_train)
        metric_values = step(x, y_true, training=True)
        metric_util.update(metric_values, training=True)
    
    model.save_weights(checkdir+'/weights.%03i.h5' % (k+1,))

    for i in tqdm(range(gen_val.num_batches), 'validation', leave=False):
        x, y_true = next(iterator_val)
        metric_values = step(x, y_true, training=False)
        metric_util.update(metric_values, training=False)

    metric_util.on_epoch_end(verbose=1)

In [ ]:
from utils.model import calc_memory_usage, count_parameters
count_parameters(model)
calc_memory_usage(model)

In [ ]:
# frequency of class instance in local ground truth, used for weightning the focal loss
s = np.zeros(gt_util.num_classes)
for i in range(1000):#range(gt_util.num_samples):
    egt = prior_util.encode(gt_util.data[i])
    s += np.sum(egt[:,-gt_util.num_classes:], axis=0)
sn = np.asarray(np.sum(s))/s
print(np.array(sn, dtype=np.int32))
print(sn/np.sum(sn))